In [1]:
import numpy as np
import cvxpy as cvx
import scipy

(CVXPY) Oct 10 07:37:54 PM: Encountered unexpected exception importing solver OSQP:
ImportError('DLL load failed while importing qdldl: The specified module could not be found.')


Given an interaction matrix $A \in \mathbb{R}^{N \times N}$, where $N$ is the number of nodes, we seek to find the ``true" ranking of the nodes in the system.

One approach is to use SpringRank algorithm, which seeks the optimal ranks $\mathbf{s}^{*} = \begin{bmatrix} s_{1}^{*} & s_{2}^{*} & \cdots s_{N}^{*} \end{bmatrix}^{T}$ by minimizing the total energy of the system, given by the Hamiltonian, $H: \mathbb{R}^{N} \mapsto \mathbb{R}_{+}$.
Here,
\begin{align}
  H \left( \mathbf{s} \right) & = \sum_{i, j,  i \neq j}^{N} \left( \sqrt{A_{ij}} \left( s_{i} - s_{j} \right) - \sqrt{A_{ij}} \right)^{2} \\
  & = \sum_{i, j, i \neq j} A_{ij} \left( \left( s_{i} - s_{j} \right) - 1 \right)^{2},
\end{align}
where $\mathbf{s}$'s entries are candidate ranks of the nodes in the system.

Note that minimizing $H$ is equivalent to minimizing the norm of the vector $\mathbf{H} \in \mathbb{R}^{N \left( N - 1 \right)}$, where
\begin{align}
  \mathbf{H} & = A^{\frac{1}{2}} \left( L \mathbf{s} - \mathbb{1} \right).
\end{align}
Here,  
*   $\mathbb{1} \in \mathbb{R}^{N \left( N - 1 \right)}$ is the all-ones vector
*   $L: \mathbb{R}^{N} \mapsto \mathbb{R}^{N \left( N - 1 \right)}$ is the linear operator that maps $\mathbf{s}$ to the following vector:

\begin{align}
  L \mathbf{s} & = \begin{bmatrix}
  s_{1} - s_{2} \\
  s_{1} - s_{3} \\
  \vdots \\
  s_{1} - s_{N} \\
  s_{2} - s_{1} \\
  s_{2} - s_{3} \\
  \vdots \\
  s_{2} - s_{N} \\
  \vdots \\
  s_{N} - s_{1} \\
  s_{N} - s_{2} \\
  \vdots \\
  s_{N} - s_{N-1}
  \end{bmatrix}
\end{align}

*   $A^{\frac{1}{2}}: \mathbb{R}^{N \left( N - 1 \right)} \mapsto \mathbb{R}^{N \left( N - 1 \right)}$ is the linear operator that maps $\left( L \mathbf{s} - \mathbb{1} \right)$ to the following vector:
\begin{align}
  A^{\frac{1}{2}} \left( L \mathbf{s} - \mathbb{1} \right) & =
  \begin{bmatrix}
  \sqrt{A_{1,2}} \left( s_{1} - s_{2} - 1 \right) \\
  \sqrt{A_{1, 3}} \left( s_{1} - s_{3} - 1 \right) \\
  \vdots \\
  \sqrt{A_{1, N}} \left( s_{1} - s_{N} - 1 \right) \\
  \sqrt{A_{2, 1}} \left( s_{2} - s_{1} - 1 \right) \\
  \sqrt{A_{2, 3}} \left( s_{2} - s_{3} - 1 \right) \\
  \vdots \\
  \sqrt{A_{2, N}} \left( s_{2} - s_{N} - 1 \right) \\
  \vdots \\
  \sqrt{A_{N, 1}} \left( s_{N} - s_{1} - 1 \right) \\
  \sqrt{A_{N, 2}} \left( s_{N} - s_{2} - 1 \right) \\
  \vdots \\
  \sqrt{A_{N, N-1}} \left( s_{N} - s_{N-1} - 1 \right)
  \end{bmatrix}
\end{align}
Note $A^{\frac{1}{2}}$ is a slight abuse of notation, as we are not referring to a matrix square root, but the square root of the off-diagonal entries of $A$.

We represent the operators $L$ and $A$ in matrix form as follows:

\begin{align}
  L & =
  \begin{bmatrix}
  1 & -1 & 0 & 0 & \cdots & 0 & 0 \\
  1 & 0 & -1 & 0 & \cdots & 0 & 0 \\
  \vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots \\
  1 & 0 & 0 & 0 & \cdots & 0 & -1 \\
  -1 & 1 & 0 & 0 & \cdots & 0 & 0 \\
  0 & 1 & -1 & 0 & \cdots & 0 & 0 \\
  \vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots \\
  0 & 1 & 0 & 0 & \cdots & 0 & -1 &\\
  \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots \\
  -1 & 0 & 0 & 0 & \cdots & 0 & 1 \\
  0 & -1 & 0 & 0 & \cdots & 0 & 1 \\
  \vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots \\
  0 & 0 & 0 & 0 & \cdots 0 & -1 & 1
  \end{bmatrix} \in \mathbb{R}^{N \left( N - 1 \right) \times N} \\
  A^{\frac{1}{2}} & = \text{diag} \left( \sqrt{A_{1, 2}}, \sqrt{A_{1, 3}}, \dots, \sqrt{A_{2, 1}}, \sqrt{A_{2, 3}}, \dots, \sqrt{A_{2, N}}, \dots, \sqrt{A_{N, 1}}, \sqrt{A_{N, 2}}, \dots, \sqrt{A_{N, N-1}} \right) \in \mathbb{R}^{N \left( N - 1 \right) \times N \left( N - 1 \right)}.
\end{align}

Thus, $H \left( \mathbf{s} \right) = \left| \left| \mathbf{H} \right| \right|_{2}^{2}$.

We know the matrix $L$ and the all-ones vector $\mathbb{1}$ exactly.
However, there is possibly uncertainty in the interaction matrix $A$, i.e., the matrix $A$ is given by
\begin{align}
A & = \overline{A} + \Delta A,
\end{align}
where $\overline{A}$ is the true, unknown interaction matrix and $\Delta A$ is noise.
We do not know $\Delta A$ explicitly, but we can make inferences based on the problem.

This implies that there is uncertainty in $A^{\frac{1}{2}}$ as an operator.
That is,
\begin{align}
A^{\frac{1}{2}} = \left( \overline{A} \right)^{\frac{1}{2}} + \Delta A^{\frac{1}{2}},
\end{align}
where $\left( \overline{A} \right)^{\frac{1}{2}}$ is the true, unknown operator and $\Delta A^{\frac{1}{2}}$ is noise.
We do not know $\Delta A^{\frac{1}{2}}$ explicitly, but as above, we can make inferences based on the problem.

**IDEA**: We are looking to find the $\mathbf{s}$ that minimizes $H$, or equivalently, that minimizes $\left| \left| \mathbf{H} \right| \right|_{2}^{2}$.
Minimizing $\left| \left| \mathbf{H} \right| \right|_{2}^{2}$ is equivalent to the following problem,

\begin{align}
  \min_{\mathbf{s} \in \mathbb{R}^{N}} \left| \left| A^{\frac{1}{2}} \left( L \mathbf{s} - \mathbb{1} \right) \right| \right|_{2}^{2},
\end{align}
which we ought to be able to implement in CVXPY.

Other ideas:

* Making sure the problem is translation invariant, i.e., if $\mathbf{s}^{*}$ is the optimal solution, then so is $\mathbf{s}^{*} + \alpha$ for any constant $\alpha$
* Ensuring that the CVXPY approach returns as good of a solution compared to the SpringRank approach - currently, there is a discrepancy between the solutions, e.g., the solution returned by the SpringRank code is not simply a translated version of the CVXPY solution, and vice versa. However, given the same interaction matrix of nodes in a graph, both the SpringRank and CVXPY approach return nodes that have the same rankings. Might be worth asking about.
* The Hamiltonian minimization problem in the SpringRank paper might make use of the diagonal entries of the $A$ matrix?
In this case, the $L$ matrix would be of size $\mathbb{R}^{N^{2} \times N}$, but the additional rows would simply be zero, and the $A^{\frac{1}{2}}$ matrix would be of size $\mathbb{R}^{N^{2} \times N^{2}}$ as well.
Need to ask about this as well.

In [2]:
# From the SpringRank paper (A physical model for efficient ranking in networks by Bacco et al.)
# Link: https://www.science.org/doi/10.1126/sciadv.aar8260

alpha = 0.0

################################################################################
# Approach 1: Ignore the diagonal entries of the interaction matrix A
# If there are N nodes in the system, then:
# 1. L is of size (N*(N-1)) x N
# 2. A^(1/2) is of size (N*(N-1)) x (N*(N-1))
################################################################################

# Remove the diagonal entries
# A = np.array([[1, 4, 3], [3, 1, 4], [2, 2, 1]])
# A = np.array([[1, 2, 3], [2, 1, 4], [2, 2, 1]])
A = np.array([[1, 4, 3], [3, 1, 4], [2, 2, 1]])
mask = ~np.eye(A.shape[0], dtype=bool)
A_modified = A[mask]

print(A_modified)

# Create the matrix A^(1/2)
A_1_2 = scipy.sparse.diags(np.sqrt(A_modified.flatten()))

N = A.shape[0]

# Computes the L operator that is given above
# def L_op(s):
#   N = s.size
#   val = np.zeros((N * (N-1), ))

#   true_inds_i = np.repeat(np.arange(N), (N-1), axis=0)

#   for i in np.arange(N):
#     rows = np.arange(i*(N-1), (i+1)*(N-1))
#     cols = np.arange(N)
#     cols = cols[cols != i]

L = np.zeros((N * (N-1), N))

for i in np.arange(N):
  rows = np.arange(i*(N-1), (i+1)*(N-1))
  cols = np.arange(N)
  cols = cols[cols != i]
  L[rows, i] = 1
  L[rows, cols] = -1

print(L)

all_ones = np.ones((L.shape[0], ))
all_ones = cvx.vec(all_ones)

s = cvx.Variable(N)
obj = cvx.Minimize(cvx.norm(A_1_2 @ ((L @ s) - all_ones), 2)**2 + alpha * cvx.norm(s, 2)**2)
# constraints = [s[-1]==0]
constraints = []
prob = cvx.Problem(obj, constraints)
prob.solve(verbose=True)
print(f"########################################################")
print(f"Solution when A's diagonal terms are excluded: {s.value}")
print(f"Ranks when A's diagonal terms are included: {np.argsort(s.value) + 1}")
print(f"########################################################")

AA = A_1_2 @ L
G  = AA.T @ AA
print('LHS of normal eqn is:')
print(G)

################################################################################
# Approach 2: Don't ignore the diagonal entries of the interaction matrix A
# If there are N nodes in the system, then
# 1. L is of size (N*N) x N
# 2. A^(1/2) is of size (N*N) x (N*N)
################################################################################
# A = np.loadtxt("interaction-matrix.txt", dtype=int)
# A = np.array([[1, 2, 3], [2, 1, 4], [2, 2, 1]])
# A = np.array([[1, 4, 3], [3, 1, 4], [2, 2, 1]])
# A = np.array([[1, 2, 3, 4], [2, 1, 3, 5], [3, 3, 1, 5], [4, 5, 8, 1]])
A_1_2 = scipy.sparse.diags(np.sqrt(A.flatten()))

N = A.shape[0]

# Reference L operator for when N = 3
# L_true = np.array([
#     [0, 0, 0],
#     [1, -1, 0],
#     [1, 0, -1],
#     [-1, 1, 0],
#     [0, 0, 0],
#     [0, 1, -1],
#     [-1, 0, 1],
#     [0, -1, 1],
#     [0, 0, 0]
# ])

L = np.zeros((N * N, N), dtype=int)

for i in np.arange(N):
  for j in np.arange(N):
    L[N*i+j][i] += 1
    L[N*i+j][j] += -1

# print(L)

all_ones = np.ones((L.shape[0], ))
all_ones = cvx.vec(all_ones)

# print(np.linalg.norm(L_true - L))

s = cvx.Variable(N)
obj = cvx.Minimize(cvx.norm(A_1_2 @ ((L @ s) - all_ones), 2)**2 + alpha * cvx.norm(s, 2)**2)
# constraints = [ s[-1] == 0 ]
constraints = []
prob = cvx.Problem(obj, constraints)
prob.solve(verbose=True)

print(f"########################################################")
print(f"Solution when A's diagonal terms are included: {s.value}")
print(f"Ranks when A's diagonal terms are included: {np.argsort(s.value) + 1}")
print(f"########################################################")

[4 3 3 4 2 2]
[[ 1. -1.  0.]
 [ 1.  0. -1.]
 [-1.  1.  0.]
 [ 0.  1. -1.]
 [-1.  0.  1.]
 [ 0. -1.  1.]]
                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Oct 10 07:38:01 PM: Your problem has 3 variables, 0 constraints, and 0 parameters.
(CVXPY) Oct 10 07:38:01 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Oct 10 07:38:01 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Oct 10 07:38:01 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Oct 10 07:38:01 PM: Compiling problem (target solver

In [3]:
# SpringRank_ranks = np.array( [1.05928854, 1.11067194, 0.83003953])
SpringRank_ranks = np.array([1.11731844, 1.05586592, 0.82681564])
print(s.value - SpringRank_ranks)

[-0.99270785 -0.99667589 -1.01061627]


In [4]:
# Save the solution to disk
fname_cvxpy_ranks = "CVXPY-ranks.txt"
np.savetxt(fname_cvxpy_ranks, s.value, fmt="%f")

# Now, load the SpringRank reference code ranks, and see how it compares
# to the CVXPY solution
# fname_ranks = "SpringRank-ranks.txt"
# SpringRank_ranks = np.loadtxt(fname_ranks, dtype=float)

In [5]:
alpha = s.value.min()

# Check norm of H vector
H_norm = np.linalg.norm(A_1_2 @ (L @ (s.value) - 1), 2)
print(f"Norm of H vector (no shift in s): {H_norm}")

# Translate the returned solution vector, s, by a constant
# If the code works correctly, the norm of the H vector should be unchanged
H_shifted_s_norm = np.linalg.norm(A_1_2 @ (L @ (s.value - alpha) - 1), 2)
print(f"Norm of H vector (s shifted by alpha): {H_shifted_s_norm}")

# Print the norm of the H vector, but the s here has been computed by the
# SpringRank code
H_s_springrank_norm = np.linalg.norm(A_1_2 @ (L @ (SpringRank_ranks) - 1), 2)
print(f"Norm of H vector (s computed via SpringRank): {H_s_springrank_norm}")

# Print the ranks of the nodes
print(f"Sorted CVXPY ranks: {np.argsort(s.value) + 1}")
print(f"Sorted SpringRank ranks: {np.argsort(SpringRank_ranks) + 1}")

# other_s = np.array([0.42345924453280354, 0.5685884691848911, 0])
# other_s = np.array([0.49058025621703105, 0.423511680482291, 0])

# print(np.linalg.norm(A_1_2 @ ((L @ other_s) - 1)))

# print(np.argsort(s.value - s.value.min()))
# print(np.argsort(other_s))

# print(np.linalg.norm(A_1_2 @ (L @ s.value - 1)))
# print(np.linalg.norm(A_1_2 @ (L @ np.abs(s.value) - 1)))

Norm of H vector (no shift in s): 4.487781959485983
Norm of H vector (s shifted by alpha): 4.487781959485983
Norm of H vector (s computed via SpringRank): 4.488102794516646
Sorted CVXPY ranks: [3 2 1]
Sorted SpringRank ranks: [3 2 1]


Ideas:
* Complicated: if the $\left( i, j \right)$ entry of $A$ goes up by $1$, make the $\left( j, i \right)$ entry of $A$ go down by $1$
* When $\alpha > 0$, the CVXPY and SpringRank models return different rankings, but the sorted ranks are the same
When we take the difference between the two sets of ranks, there is a difference of about $1$, possibly due to solver accuracy
* Try solving the CVXPY model by assuming that there is noise in the model, and try to solve $\min_{x, s} \max_{\left| \left| \Delta \right| \right| \leq \delta} \frac{1}{2} \left| \left| \left( A^{\frac{1}{2}} + \Delta \right) x - 0 \right| \right|_{2}^{2}$, with the constraint that $x = Ls - 1$
This (might?) let us use the robust least squares framework developed in the Clancy and Becker paper, and the constraint that $x = Ls - 1$ ensures that $x$ is in the range of $L$ ($L$ is not full rank, as the all-ones vector is a non-trivial element of $\text{ker} \left( L \right)$)
* Here, the $\delta$'s ought to be informed by the underlying data
* For the time being, try with a fixed $\delta$, e.g., $\delta = 1$
* Another idea to try is to have a generative model, i.e., we set the ground truth, but add noise to it, and draw models from the distribution

In [16]:
# Try to solve the problem with a fixed delta
delta = 0.01

s2 = cvx.Variable(N)
x2 = cvx.Variable(N*N)
D  = delta * np.outer(np.ones((N*N, )), np.ones((N*N, )))
all_ones = cvx.vec(np.ones((N*N, )))

obj2 = cvx.Minimize(cvx.norm(cvx.abs(A_1_2 @ x2) + D @ cvx.abs(x2)))
constraints2 = [x2 == ((L @ s2) - all_ones)] # Ensure that x2 is in the range of L
prob2 = cvx.Problem(obj2, constraints2)
prob2.solve(verbose=True)

print(x2.value)
print(s2.value)
print(f"{np.argsort(s2.value) + 1}")

                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Oct 10 08:16:47 PM: Your problem has 12 variables, 1 constraints, and 0 parameters.
(CVXPY) Oct 10 08:16:47 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Oct 10 08:16:47 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Oct 10 08:16:47 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Oct 10 08:16:47 PM: Compiling problem (target solver=ECOS).
(CVXPY) Oct 10 08:16:47 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing ->